In [1]:
!unzip  -qO UTF-8 data/data211908/地铁数据.zip -d ./data
!pip install --upgrade pip
!pip install paddlets

replace ./data/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 2.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 22.1.2
    Uninstalling pip-22.1.2:
      Successfully uninstalled pip-22.1.2
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.6/572.6 kB 1.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 1.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 2.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 1.5 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 2.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 2.0 MB/s eta 0:00:0000:0100:01
    

In [2]:
import re
import pandas as pd

df = pd.read_csv("data/客流量数据.csv")

def extract_station(x):
    station = re.compile("([A-Z])站").findall(x)
    if len(station)>0:
        return station[0]

df["时间区段"] = df["时间区段"].apply(lambda x:x.split("-")[0])
df["站点"] = df["站点"].apply(extract_station)
df["时间"] = df["时间"].apply(lambda x:str(x)).str.cat(df['时间区段'],sep=" ")
df["时间"] = pd.to_datetime(df["时间"])
df = df.drop("时间区段",axis=1)
df.columns =["Time", "Station", "InNum", "OutNum"] 
df.head()

,Time,Station,InNum,OutNum
0,2022-01-01,G,63,21
1,2022-01-01,F,42,84
2,2022-01-01,E,0,105
3,2022-01-01,D,42,126
4,2022-01-01,C,42,105


In [ ]:
import paddle
from paddlets.datasets.tsdataset import TSDataset
from paddlets.transform import TimeFeatureGenerator, StandardScaler
from paddlets.models.forecasting import LSTNetRegressor
from paddlets.metrics import MAE
import warnings
warnings.filterwarnings('ignore')

stations=["A","B","C","D","E","F","G"]
for station in stations:
    # station = "A" # 站点
    dataset_df = df[df['Station']==station]
    dataset_df.head()
    dataset_df = TSDataset.load_from_dataframe(
    dataset_df,
    time_col='Time', #时间序列
    target_cols=['InNum','OutNum'], #预测目标
    freq='15min',
    fill_missing_dates=True,
    fillna_method='zero'
    )

    dataset_df.plot()

    dataset_df.summary()

    # 训练集和验证集
    dataset_train, dataset_val_test = dataset_df.split("2023-02-05 23:45:00")
    # 测试集
    dataset_val, dataset_test = dataset_val_test.split("2023-03-29 23:45:00")

    # 数据标准化
    scaler = StandardScaler()
    scaler.fit(dataset_train)
    dataset_train_scaled = scaler.transform(dataset_train)
    dataset_val_test_scaled = scaler.transform(dataset_val_test)
    dataset_val_scaled = scaler.transform(dataset_val)
    dataset_test_scaled = scaler.transform(dataset_test)

    # 模型配置定义
    paddle.seed(2023)
    model = LSTNetRegressor(
        in_chunk_len = 4*24,
        out_chunk_len = 4*24,
        max_epochs = 200,
        patience = 20,
        eval_metrics =["mae"],
        optimizer_params= dict(learning_rate=3e-3)
    )

    # 开始模型训练
    model.fit(dataset_train_scaled, dataset_val_scaled)

    # 模型训练完后，用测试集的数据对模型的效果进行回测
    from paddlets.utils import backtest
    from paddlets.metrics import MAE
    mae, pred = backtest(data=dataset_test_scaled,
                    model=model,
                    metric=MAE(),
                    return_predicts=True
    )

    _, ground_truth = dataset_test_scaled.split("2023-03-30 23:45:00")

    pred.plot(add_data = ground_truth)

    mae

    # 将数据反标准化
    ground_truth_inverse = scaler.inverse_transform(ground_truth)
    pred_inverse = scaler.inverse_transform(pred)
    pred_inverse.plot(add_data=ground_truth_inverse)

    #模型保存
    import os
    import pickle

    model_save_path = os.path.join("models/lstm",station)
    if not os.path.exists(model_save_path):
        os.makedirs(model_save_path)
    model.save(os.path.join(model_save_path, "model"))
    pickle.dump(scaler, open(os.path.join(model_save_path,"scaler.pkl"),'wb'))

    # 模型推理
    # 先将测试数据集读入，转化为TSDataset类型。
    import pandas as pd
    from paddlets.datasets.tsdataset import TSDataset
    from paddlets.transform import TimeFeatureGenerator, StandardScaler

    dataset_test_df = pd.read_csv("data/test.csv")
    # station = "A" # 站点
    dataset_test_df = dataset_test_df[dataset_test_df['Station']==station]

    dataset_test_df = TSDataset.load_from_dataframe(
        dataset_test_df,
        time_col='Time',
        target_cols=['InNum','OutNum'],
        freq='15min',
        fill_missing_dates=False,
    )

    dataset_test_df

    #加载标准化处理参数，对测试数据进行标准化预处理。
    import pickle
    with open("models/lstm/{}/scaler.pkl".format(station), "rb") as r:
        scaler = pickle.load(r)
    dataset_test_scaled = scaler.transform(dataset_test_df)
    dataset_test_scaled

    #将保存好的模型重新加载，调用.predict函数对测试数据进行预测，对预测结果进行反标准化，再处理负值和浮点型问题即可输出为结果文件。
    import os
    from paddlets.models.model_loader import load

    model = load("models/lstm/{}/model".format(station))
    res = model.predict(dataset_test_scaled)
    res_inversed = scaler.inverse_transform(res)
    res_inversed = res_inversed.to_dataframe(copy=True)
    res_inversed["InNum"] = res_inversed["InNum"].apply(lambda x:0 if x<0 else int(x))
    res_inversed["OutNum"] = res_inversed["OutNum"].apply(lambda x:0 if x<0 else int(x))
    res_inversed["Station"] = station # 添加站点标识
    res_inversed.index.name = "Time"
    # 输出结果文件
    if not os.path.exists("./results"):
        os.makedirs("./results")
    res_inversed.to_csv("results/{}_result.csv".format(station))
    res_inversed.head()

# 合并结果
import pandas as pd

df1=pd.read_csv("results/A_result.csv")
df2=pd.read_csv("results/B_result.csv")
df3=pd.read_csv("results/C_result.csv")
df4=pd.read_csv("results/D_result.csv")
df5=pd.read_csv("results/E_result.csv")
df6=pd.read_csv("results/F_result.csv")
df7=pd.read_csv("results/G_result.csv")

merged_df=pd.concat([df1,df2,df3,df4,df5,df6,df7])

merged_df.head()

merged_df.to_csv("results/merged_result.csv", index=False)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlets/utils/backtest.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import defaultdict, Iterable
[2023-05-31 01:07:54,359] [paddlets.models.common.callbacks.callbacks] [INFO] epoch 000| loss: 0.301634| val_0_mae: 0.461797| 0:00:16s
[2023-05-31 01:08:11,141] [paddlets.models.common.callbacks.callbacks] [INFO] epoch 001| loss: 0.261259| val_0_mae: 0.464089| 0:00:33s
[2023-05-31 01:08:27,897] [paddlets.models.common.callbacks.callbacks] [INFO] epoch 002| loss: 0.260089| val_0_mae: 0.458437| 0:00:50s
[2023-05-31 01:08:44,563] [paddlets.models.common.callbacks.callbacks] [INFO] epoch 003| loss: 0.259470| val_0_mae: 0.453110| 0:01:07s
[2023-05-31 01:09:01,179] [paddlets.models.common.callbacks.callbacks] [INFO] epoch 004| loss: 0.259350| val_0_mae: 0.459568| 0:01:23s
[2023-05-31 01:09:17,971] [paddl